# Create a histogram of agent attributes

**Author: Lukas Breitwieser**<br>
In this tutorial we will show how to create a histogram of all agent diameters in the simulation and fit a function to the data.

Let's start by setting up BioDynaMo notebooks.

In [ ]:
%jsroot on
gROOT->LoadMacro("${BDMSYS}/etc/rootlogon.C");

We want to define a function that creates a cell at a certain position with diameters drawn from a gaussian distribution with $\mu=20$ and $\sigma=5$.
The smallest diameter should be larger then $2.0$.

In [ ]:
simulation.GetResourceManager()->ClearAgents();
auto rng = simulation.GetRandom()->GetGausRng(20, 5);
auto create_cell = [&](const Double3& position) {
  Cell* cell = new Cell(position);
  double diameter = std::max(2.0, rng.Sample());
  cell->SetDiameter(diameter);
  return cell;
};

Now that we defined `create_cell` we can use it to create 400 cells on a plane with $z = 0$, $xmin = ymin = -200$, $xmax = ymax = 200$, and spacing = 20 in both dimensions.

In [ ]:
auto f = [](const double* x, const double* params) { return 0.0; };
ModelInitializer::CreateAgentsOnSurface(f, {}, -200, 200, 20, -200, 200, 20, 
                                        create_cell);
simulation.GetScheduler()->FinalizeInitialization();
VisualizeInNotebook(300, 300);

The next step is to create a histogram object with 100 bins in the interval [2, 40].<br>
The second line creates a function which fills the histogram with the diameter of the given agent.<br>
The third line calls the function `fill` for each agent, thus adding all diameters to the histogram.

In [ ]:
TH1F h("myHisto","Agent Diameter Histogram;Diameter;Count", 100, 2, 40);
auto fill = L2F([&](Agent* a, AgentHandle){ h.Fill(a->GetDiameter()); });
simulation.GetResourceManager()->ForEachAgent(fill);

Let's draw the final histogram.<br>
Before we have to create a `TCanvas` object in order to display the result in this notebook.<br>
We also modify the default color and create a grid.

In [ ]:
TCanvas c("", "", 400, 300);
h.SetFillColor(kBlue - 10);
c.SetGrid();
h.Draw();
c.Draw();

Finally, we can try to fit a function to the data in the histogram.<br>
Since we drew samples from a gaussian random number generator when we created our cells, we expect that a gaussian will fit our data.

In [ ]:
h.Fit("gaus", "S");
h.Draw();
c.Draw();